# **The Next Hot Place to Live in Brooklyn** 

I will try to figure out the next tendy neighborhood in brooklyn by comparing them to them to current trendy areas such as Williamsburg and Park Slope. Then I will find the area that compares to them and has the lowest rent. 

In [1]:
import bs4 as bs

import urllib.request  

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


In [2]:
brooklyn_rent_file='brooklyn.csv'
rent_bk=pd.read_csv(brooklyn_rent_file)
rent_bk

,Neighborhood,Average Rent
0,Flatlands,1492
1,Marine Park,1659
2,Madison,1659
3,Gerritsen Beach,1659
4,Homecrest,1717
5,Bensonhurst,1795
6,Bath Beach,1795
7,Midwood,1825
8,Fiske Terrace,1825
9,Kensington,1839


check columns

In [3]:
for col in rent_bk.columns: 
    print(col) 

Neighborhood
Average Rent


Get brooklyn data

In [4]:
with open('nyu-2451-34572-geojson.json') as json_data:
    new_york_data = json.load(json_data)

In [5]:
neighborhoods_data = new_york_data['features']

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()


,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [10]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [11]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'],brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

FourSquare Data

In [12]:
CLIENT_ID = 'UAU4F0LIK3WOOWV5RCM5Z2PCCYTN402X42QVDHNCOZJYEPBS' 
CLIENT_SECRET = '5DJPNJ2C2DMW0HYAMP1Y0KXDRONEQYDFQCWMAMG2LT0M4KTG' 
VERSION = '20180604'
LIMIT = 500

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


Check Size

In [17]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2680, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
4,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


Check how many venues were returned for each neighborhood



In [18]:
brooklyn_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,48,48,48,48,48,48
Bay Ridge,79,79,79,79,79,79
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,31,31,31,31,31,31
Bergen Beach,7,7,7,7,7,7
Boerum Hill,91,91,91,91,91,91
Borough Park,16,16,16,16,16,16
Brighton Beach,45,45,45,45,45,45
Broadway Junction,17,17,17,17,17,17


In [19]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))


There are 282 uniques categories.


Analyze each area

In [20]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Resta

In [21]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Resta

In [22]:
num_top_venues = 10

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                       venue  freq
0                   Pharmacy  0.06
1                Gas Station  0.04
2         Chinese Restaurant  0.04
3       Fast Food Restaurant  0.04
4         Italian Restaurant  0.04
5               Dessert Shop  0.04
6            Bubble Tea Shop  0.04
7           Sushi Restaurant  0.02
8  Middle Eastern Restaurant  0.02
9      Check Cashing Service  0.02


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.09
1                  Spa  0.06
2          Pizza Place  0.05
3                  Bar  0.04
4     Greek Restaurant  0.04
5  American Restaurant  0.04
6       Sandwich Place  0.03
7           Bagel Shop  0.03
8       Ice Cream Shop  0.03
9        Grocery Store  0.03


----Bedford Stuyvesant----
                     venue  freq
0                     Café  0.07
1                      Bar  0.07
2                 Bus Stop  0.07
3              Coffee Shop  0.07
4              Pizza Place  0.07
5              Bus Station  0.

                  venue  freq
0            Print Shop  0.08
1              Pharmacy  0.08
2           Supermarket  0.08
3         Moving Target  0.08
4     Food & Drink Shop  0.08
5  Caribbean Restaurant  0.08
6  Fast Food Restaurant  0.08
7    Chinese Restaurant  0.08
8        Hardware Store  0.08
9                  Park  0.08


----East New York----
                       venue  freq
0              Deli / Bodega  0.24
1         Spanish Restaurant  0.12
2        Fried Chicken Joint  0.06
3           Asian Restaurant  0.06
4       Fast Food Restaurant  0.06
5         Chinese Restaurant  0.06
6       Caribbean Restaurant  0.06
7              Event Service  0.06
8              Metro Station  0.06
9  Latin American Restaurant  0.06


----East Williamsburg----
                venue  freq
0                 Bar  0.08
1       Deli / Bodega  0.08
2        Cocktail Bar  0.06
3              Bakery  0.06
4         Coffee Shop  0.06
5         Music Venue  0.04
6        Concert Hall  0.04
7  Mexica



----New Lots----
                 venue  freq
0          Pizza Place  0.09
1  Fried Chicken Joint  0.09
2   Spanish Restaurant  0.09
3        Grocery Store  0.09
4       Discount Store  0.05
5       Breakfast Spot  0.05
6   Chinese Restaurant  0.05
7                 Park  0.05
8        Metro Station  0.05
9             Pharmacy  0.05


----North Side----
                           venue  freq
0                    Coffee Shop  0.10
1                    Yoga Studio  0.05
2                    Pizza Place  0.05
3                         Bakery  0.04
4            American Restaurant  0.04
5                            Bar  0.04
6                       Wine Bar  0.03
7  Vegetarian / Vegan Restaurant  0.03
8                  Jewelry Store  0.03
9                   Cocktail Bar  0.02


----Ocean Hill----
                             venue  freq
0                    Deli / Bodega  0.17
1                       Playground  0.08
2                Convenience Store  0.08
3  Southern / Soul Food Res

Top 5 most common venues above, below put in pandas data frame

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Bubble Tea Shop,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Gas Station,Dessert Shop,Flower Shop,Hookah Bar,Donut Shop
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,American Restaurant,Bar,Greek Restaurant,Bagel Shop,Sandwich Place,Playground,Hookah Bar
2,Bedford Stuyvesant,Bar,Bus Stop,Pizza Place,Café,Coffee Shop,Juice Bar,Bus Station,Boutique,New American Restaurant,Gift Shop
3,Bensonhurst,Bakery,Sushi Restaurant,Grocery Store,Italian Restaurant,Ice Cream Shop,Donut Shop,Chinese Restaurant,Noodle House,Russian Restaurant,Liquor Store
4,Bergen Beach,Harbor / Marina,Donut Shop,Playground,Park,Athletics & Sports,Baseball Field,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant
5,Boerum Hill,Coffee Shop,Bar,Dance Studio,Sandwich Place,Spa,Arts & Crafts Store,Bakery,Gym / Fitness Center,Furniture / Home Store,Cocktail Bar
6,Borough Park,Bank,Pizza Place,Pharmacy,Hotel,Coffee Shop,Restaurant,Café,Fast Food Restaurant,Deli / Bodega,American Restaurant
7,Brighton Beach,Beach,Russian Restaurant,Restaurant,Eastern European Restaurant,Gourmet Shop,Sushi Restaurant,Bank,Pharmacy,Mobile Phone Shop,Diner
8,Broadway Junction,Fried Chicken Joint,Donut Shop,Diner,Dessert Shop,Burger Joint,Nightclub,Metro Station,Gas Station,Caribbean Restaurant,Sandwich Place
9,Brooklyn Heights,Yoga Studio,Deli / Bodega,Park,Italian Restaurant,Pizza Place,Bakery,Pharmacy,Cosmetics Shop,Gym,Mexican Restaurant


Cluster Neighborhoods

In [25]:
# set number of clusters
kclusters = 4

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0])

In [26]:
brooklyn_merged = brooklyn_data

# add clustering labels
brooklyn_merged['Cluster Labels'] = kmeans.labels_

brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head(15) # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Italian Restaurant,Spa,Pizza Place,American Restaurant,Bar,Greek Restaurant,Bagel Shop,Sandwich Place,Playground,Hookah Bar
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0,Bakery,Sushi Restaurant,Grocery Store,Italian Restaurant,Ice Cream Shop,Donut Shop,Chinese Restaurant,Noodle House,Russian Restaurant,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Bank,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Gym,Fried Chicken Joint,Mobile Phone Shop,Women's Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Yoga Studio,Sushi Restaurant,Café,Restaurant,Polish Restaurant,New American Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Bus Station,Deli / Bodega,Lounge,Bakery,Italian Restaurant,Chinese Restaurant,Jewelry Store,Bar,Electronics Store
5,Brooklyn,Brighton Beach,40.576825,-73.965094,0,Beach,Russian Restaurant,Restaurant,Eastern European Restaurant,Gourmet Shop,Sushi Restaurant,Bank,Pharmacy,Mobile Phone Shop,Diner
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,3,Dessert Shop,Turkish Restaurant,Sandwich Place,Bagel Shop,Diner,Creperie,Outlet Store,Restaurant,Russian Restaurant,Café
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,0,Pizza Place,Donut Shop,Ice Cream Shop,Jazz Club,Liquor Store,Chinese Restaurant,Restaurant,Steakhouse,Bank,Grocery Store
8,Brooklyn,Flatbush,40.636326,-73.958401,0,Juice Bar,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Coffee Shop,Donut Shop,Bagel Shop,Grocery Store,Chinese Restaurant,Lounge
9,Brooklyn,Crown Heights,40.670829,-73.943291,0,Pizza Place,Museum,Café,Grocery Store,Playground,Supermarket,Sushi Restaurant,Candy Store,Salon / Barbershop,Electronics Store


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Look at all the clusters

Cluster 1

In [28]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Pizza Place,American Restaurant,Bar,Greek Restaurant,Bagel Shop,Sandwich Place,Playground,Hookah Bar
1,Bensonhurst,Bakery,Sushi Restaurant,Grocery Store,Italian Restaurant,Ice Cream Shop,Donut Shop,Chinese Restaurant,Noodle House,Russian Restaurant,Liquor Store
2,Sunset Park,Bank,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Gym,Fried Chicken Joint,Mobile Phone Shop,Women's Store
3,Greenpoint,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Yoga Studio,Sushi Restaurant,Café,Restaurant,Polish Restaurant,New American Restaurant
4,Gravesend,Pizza Place,Bus Station,Deli / Bodega,Lounge,Bakery,Italian Restaurant,Chinese Restaurant,Jewelry Store,Bar,Electronics Store
5,Brighton Beach,Beach,Russian Restaurant,Restaurant,Eastern European Restaurant,Gourmet Shop,Sushi Restaurant,Bank,Pharmacy,Mobile Phone Shop,Diner
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Jazz Club,Liquor Store,Chinese Restaurant,Restaurant,Steakhouse,Bank,Grocery Store
8,Flatbush,Juice Bar,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Coffee Shop,Donut Shop,Bagel Shop,Grocery Store,Chinese Restaurant,Lounge
9,Crown Heights,Pizza Place,Museum,Café,Grocery Store,Playground,Supermarket,Sushi Restaurant,Candy Store,Salon / Barbershop,Electronics Store
11,Kensington,Thai Restaurant,Grocery Store,Sandwich Place,Pizza Place,Ice Cream Shop,Spa,Liquor Store,Supermarket,Mexican Restaurant,Burger Joint


Cluster 2

In [29]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,Caribbean Restaurant,Fast Food Restaurant,Grocery Store,Pizza Place,Mobile Phone Shop,Latin American Restaurant,Fried Chicken Joint,Mexican Restaurant,Kids Store,Pharmacy


Cluster 3

In [30]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,East Flatbush,Print Shop,Pharmacy,Caribbean Restaurant,Food & Drink Shop,Chinese Restaurant,Park,Fast Food Restaurant,Hardware Store,Supermarket,Moving Target
19,Cobble Hill,Pizza Place,Bar,Playground,Coffee Shop,Italian Restaurant,Deli / Bodega,Cocktail Bar,Yoga Studio,Middle Eastern Restaurant,Bakery
25,Cypress Hills,Fried Chicken Joint,Latin American Restaurant,Pizza Place,Ice Cream Shop,Donut Shop,Chinese Restaurant,Fast Food Restaurant,Metro Station,Spanish Restaurant,Mexican Restaurant
38,Clinton Hill,Italian Restaurant,Pizza Place,Thai Restaurant,Wine Shop,Mexican Restaurant,Indian Restaurant,Chinese Restaurant,Grocery Store,Japanese Restaurant,Restaurant
44,City Line,Donut Shop,Mobile Phone Shop,Bakery,Shoe Store,Grocery Store,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint,Mexican Restaurant,South American Restaurant
48,Georgetown,Bank,Pharmacy,Breakfast Spot,Donut Shop,Coffee Shop,Miscellaneous Shop,Burger Joint,Mexican Restaurant,Shipping Store,Frozen Yogurt Shop
50,North Side,Coffee Shop,Yoga Studio,Pizza Place,American Restaurant,Bakery,Bar,Wine Bar,Vegetarian / Vegan Restaurant,Jewelry Store,Mexican Restaurant
66,Homecrest,Bank,Grocery Store,Donut Shop,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Sandwich Place,Pizza Place,Russian Restaurant
69,Erasmus,Caribbean Restaurant,Yoga Studio,Gym / Fitness Center,Playground,Pizza Place,Convenience Store,Pharmacy,Donut Shop,Bus Line,Music Venue


Cluster 4

In [31]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Bagel Shop,Diner,Creperie,Outlet Store,Restaurant,Russian Restaurant,Café
12,Windsor Terrace,Diner,Grocery Store,Café,Deli / Bodega,Park,Bagel Shop,Chinese Restaurant,Beer Store,Coffee Shop,Bookstore
17,Bedford Stuyvesant,Bar,Bus Stop,Pizza Place,Café,Coffee Shop,Juice Bar,Bus Station,Boutique,New American Restaurant,Gift Shop
24,Park Slope,Burger Joint,Coffee Shop,Bagel Shop,Korean Restaurant,Pub,Pizza Place,Pet Store,Thai Restaurant,Bakery,Bookstore
27,Starrett City,Moving Target,Pizza Place,Bus Stop,Donut Shop,Pharmacy,American Restaurant,Caribbean Restaurant,Food,Food & Drink Shop,Flower Shop
28,Canarsie,Gym,Bus Line,Chinese Restaurant,Caribbean Restaurant,Asian Restaurant,Grocery Store,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
29,Flatlands,Pharmacy,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Lounge,Bar,Electronics Store,Paper / Office Supplies Store,Park,Nightclub
56,Rugby,Caribbean Restaurant,Grocery Store,Bank,Seafood Restaurant,Sandwich Place,Pizza Place,Wings Joint,Salon / Barbershop,Diner,Mobile Phone Shop
58,New Lots,Pizza Place,Spanish Restaurant,Grocery Store,Fried Chicken Joint,Chinese Restaurant,Breakfast Spot,Salon / Barbershop,Latin American Restaurant,Furniture / Home Store,Bank
59,Paerdegat Basin,Gym,Asian Restaurant,Moving Target,Food,Harbor / Marina,Bus Line,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


Park Slope in cluster 4 and Williamsburg in cluster 1

In [32]:
cluster4 = brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3][['Neighborhood']]
brooklyn_cluster4 = cluster4.loc[cluster4['Neighborhood'] != 'Park Slope'].reset_index(drop=True)
print("The similar neighborhoods in Brooklyn are:")
brooklyn_cluster4

The similar neighborhoods in Brooklyn are:


,Neighborhood
0,Sheepshead Bay
1,Windsor Terrace
2,Bedford Stuyvesant
3,Starrett City
4,Canarsie
5,Flatlands
6,Rugby
7,New Lots
8,Paerdegat Basin
9,Weeksville


now lets get the area with the lowest rent

In [33]:
rent_bk

,Neighborhood,Average Rent
0,Flatlands,1492
1,Marine Park,1659
2,Madison,1659
3,Gerritsen Beach,1659
4,Homecrest,1717
5,Bensonhurst,1795
6,Bath Beach,1795
7,Midwood,1825
8,Fiske Terrace,1825
9,Kensington,1839


In [34]:
pd.concat([brooklyn_cluster4, rent_bk], axis=1, join='inner')
cluster4_rent = pd.merge(brooklyn_cluster4, rent_bk,  how='left', left_on=['Neighborhood'], right_on = ['Neighborhood']).dropna()
cluster4_rent_asc = cluster4_rent.sort_values('Average Rent', axis=0, ascending = 'True').reset_index(drop=True)
cluster4_rent_asc

,Neighborhood,Average Rent
0,Flatlands,1492.0
1,Sheepshead Bay,2131.0
2,Canarsie,2178.0
3,Windsor Terrace,2361.0


Flatlands is the 1st contender, now for cluster 1

In [35]:
cluster1 = brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0][['Neighborhood']]
brooklyn_cluster1 = cluster1.loc[cluster1['Neighborhood'] != 'Williamsburg'].reset_index(drop=True)
print("The similar neighborhoods in Brooklyn are:")
brooklyn_cluster1

The similar neighborhoods in Brooklyn are:


,Neighborhood
0,Bay Ridge
1,Bensonhurst
2,Sunset Park
3,Greenpoint
4,Gravesend
5,Brighton Beach
6,Manhattan Terrace
7,Flatbush
8,Crown Heights
9,Kensington


In [36]:
pd.concat([brooklyn_cluster1, rent_bk], axis=1, join='inner')
cluster1_rent = pd.merge(brooklyn_cluster1, rent_bk,  how='left', left_on=['Neighborhood'], right_on = ['Neighborhood']).dropna()
cluster1_rent_asc = cluster1_rent.sort_values('Average Rent', axis=0, ascending = 'True').reset_index(drop=True)
cluster1_rent_asc

,Neighborhood,Average Rent
0,Madison,1659.0
1,Marine Park,1659.0
2,Gerritsen Beach,1659.0
3,Bath Beach,1795.0
4,Bensonhurst,1795.0
5,Midwood,1825.0
6,Kensington,1839.0
7,Gravesend,1840.0
8,Crown Heights,1853.0
9,Borough Park,1898.0


We can see that Madison has the lowest rent in cluster 1. The 2nd contender. 

In [37]:
flatlands = brooklyn_data.loc[29,'Neighborhood']

In [38]:
madison = brooklyn_data.loc[68,'Neighborhood']

Lets display them on a map

In [39]:
#hotplace_df = brooklyn_data.drop('Neighborhood' != 'Madison' or 'Neighborhood' != 'Flatlands', axis = True)
trendy_df = brooklyn_data[brooklyn_data.Neighborhood == 'Madison']
trendy_df = trendy_df.append(brooklyn_data[brooklyn_data.Neighborhood == 'Flatlands'])


trendy_df.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
68,Brooklyn,Madison,40.609378,-73.948415,0
29,Brooklyn,Flatlands,40.630446,-73.929113,3


In [40]:
new_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(trendy_df['Latitude'], trendy_df['Longitude'],trendy_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(new_map)  
    
new_map

**Look back at previous data:**

----Flatlands----
                           venue  freq
0                       Pharmacy  0.16
1           Fast Food Restaurant  0.11
2           Caribbean Restaurant  0.11
3            Fried Chicken Joint  0.11
4  Paper / Office Supplies Store  0.05
5                           Park  0.05
6                      Nightclub  0.05
7                            Bar  0.05
8                         Lounge  0.05
9                   Intersection  0.05




----Madison----
                venue  freq
0       Deli / Bodega  0.17
1          Bagel Shop  0.17
2          Hobby Shop  0.08
3      Pilates Studio  0.08
4         Pizza Place  0.08
5        Dessert Shop  0.08
6         Candy Store  0.08
7         Bus Station  0.08
8                 Spa  0.08
9  Italian Restaurant  0.08




You could see that Flatlands has more exciting venues such as Nightclub, Bar, and Lounge, while Madison has more choice based on food venues. I would make sure to present this data to the investor and while very subjective Flatlands seems like the more exciting Neighborhood.

# Summary: 


This project found that the closest related neighborhood to Park Slope with the lowest possible rent was Flatlands (rent: 1492.0 per month) 
and the closest related neighborhood to Williamsburg with lowest possible rent was Madison (rent:1659.0 per month) 
After mapping the two areas, we see that they are in similar locations, but Flatlands has the lower rent by far. 
This Neighborhood could experience the success that Park Slope experienced in the future, as Hipsters and Yuppies get priced out of Park Slope, Flatlands could be their next destination. Thus invesment in Flatlands could be suggested. 
